### Imports

In [ ]:
from dataclasses import dataclass
import datetime as dt
import numpy as np
import os
import h5py
import matplotlib.pyplot as plt
import pandas as pd
import json
import random
import sklearn
from tqdm import tqdm
import copy
import plotly.express as px
from collections import Counter

%matplotlib inline

In [ ]:
from torch import nn
import torch

In [ ]:
import git
repo = git.Repo(search_parent_directories=True)
sha = repo.head.object.hexsha
sha

In [ ]:
import torch
seed = 34
torch.manual_seed(seed)
random.seed(seed)
np.random.seed(seed)

In [ ]:
from joblib import Memory
location = os.path.abspath('./cachedir')
print(f'cache location: {location}')
memory = Memory(location, verbose=0)

In [ ]:
plt.rcParams['figure.figsize'] = [12, 6]

### Load Data

In [ ]:
ROOT_DATA_PATH = '/media/data/local/eurocrops/m1615987/'
H5_FILE_PATH = os.path.join(ROOT_DATA_PATH, 'HDF5s/train/AT_T33UWP_train.h5')

In [ ]:
NUMBER_OF_CHANNELS = 13

In [ ]:
def _load_data_from_h5_file(h5_file_path):
    hdf = pd.HDFStore(h5_file_path, mode='r') #here we directly load the .h5 file in one go using pandas.
    region_names = hdf.keys()  #list all the keys or regions in the region (for eg- AT112)
    region_names = region_names #[:3] - potentially limit amount of data
    df_datas = []

    for region_name in tqdm(region_names):
        df_data_single = hdf.get(f'/{region_name}') #selecting a region from based on the key (AT112 for eg.)
        df_datas.append(df_data_single)
    
    #len(set.union(*[set(x.columns) for x in df_datas]))  120 columns now, but for one region there are only 80, intesection 44
    #len(set.intersection(*[set(x.columns) for x in df_datas]))  # 120 columns now, but for one region there are only 80
    
    return df_datas, region_names
        

def _is_column_in_row_inalid(rc):
    # all_zeros = not np.any(rc) - old version - all values 0
    # new version - 2 interesting values 0 (it happens sometimes that only one channel is not zero)
    return rc[4-1] == 0 and rc[8-1] == 0  
    
    
def _find_closest_non_zero_column(time_index, common_days, dates_list, row):
    # this time step is zero, we needto find another one that is not zero.
    # To do it, find all closest non-zero columns (for all time steps for this row) 
        
    time_distance_to_nonzero_columns = [abs(common_days[time_index] - v) for v in dates_list]
    for k in range(len(time_distance_to_nonzero_columns)):
        #if not np.any(row.iloc[k]):
        if _is_column_in_row_inalid(row.iloc[k]):
            time_distance_to_nonzero_columns[k] = 9999
    closest_nonzero_column = np.argmin(time_distance_to_nonzero_columns)
    return closest_nonzero_column

    
def _resample_and_concatenate_regions_data(df_datas, resampled_days_interval):
    # Conatenation of data with different dates - fixed interval span, with finding closes date (better to use interpolation, but not with nois cloud data)
    DI = resampled_days_interval  # days interval
    common_days = list(range(DI, 365, DI))
    print(f'len(common_days) = {len(common_days)}')
    # common_days_datetime = [for day in common_days]

    # year = int(timesteps[10][:4])
    # new_year_day = dt.datetime(year=year, month=1, day=1)
    # dates_list = [((dt.datetime.strptime(date, tf)- new_year_day).days + 1) for date in timesteps]

    df_data_all = pd.DataFrame(columns=common_days)


    for df_data_single in tqdm(df_datas):
        timesteps = list(df_data_single.columns)
        year = int(timesteps[10][:4])
        new_year_day = dt.datetime(year=year, month=1, day=1)
        tf = '%Y%m%d'
        dates_list = [((dt.datetime.strptime(date, tf)- new_year_day).days + 1) for date in timesteps]
        df_data_single = df_data_single.rename(columns={old: new for old, new in zip(timesteps, dates_list)})

        closest_columns = []
        for common_day in common_days:
            closest_column = np.argmin([abs(common_day - v) for v in dates_list])
            closest_columns.append(closest_column)

        new_frames = []
        for index, row in df_data_single.iterrows():
            resampled_row_data = []
            
            for i, closest_column in enumerate(closest_columns):
                rc = row.iloc[closest_column]
                invalid_rc = _is_column_in_row_inalid(rc)
                if invalid_rc:
                    closest_nonzero_column = _find_closest_non_zero_column(
                        time_index=i,
                        common_days=common_days,
                        dates_list=dates_list, 
                        row=row)
                    rc = row.iloc[closest_nonzero_column]
                
                resampled_row_data.append(rc)

            resampled_row_df = pd.DataFrame([resampled_row_data], columns=common_days, index=[index])
            new_frames.append(resampled_row_df)

        new_frames_df = pd.concat(new_frames)
        df_data_all = pd.concat([df_data_all, new_frames_df])
    
    return df_data_all, common_days


def _load_all_labels(region_names):
    df_labels_all_lists = []
    for region_name in region_names:
        region_name = region_name.strip('/')
        LABELS_CSV_FILE_PATH = os.path.join(ROOT_DATA_PATH, f'csv_labels/train/demo_eurocrops_{region_name}.csv')
        GEO_JSON_FILE_PATH = os.path.join(ROOT_DATA_PATH, f'GeoJSONs_regional_split/train/AT/demo_eurocrops_{region_name}.geojson')

        # csv_file_path = os.path.join(train_csv_dir, csv_file_name)
        df_labels = pd.read_csv(LABELS_CSV_FILE_PATH, index_col=0)
        df_labels_all_lists.append(df_labels)


    df_labels_all = pd.concat(df_labels_all_lists)
    return df_labels_all
    

@memory.cache
def load_all_data_from_file_resampled(
        h5_file_path: str, 
        resampled_days_interval: int,
        ):
    df_datas, region_names = _load_data_from_h5_file(h5_file_path=h5_file_path)
    df_data_all, common_days = _resample_and_concatenate_regions_data(df_datas=df_datas, resampled_days_interval=resampled_days_interval)
    
    df_labels_all = _load_all_labels(region_names=region_names)

    return df_data_all, df_labels_all, common_days, region_names

In [ ]:
df_data_all, df_labels_all, common_days, region_names = load_all_data_from_file_resampled(
    h5_file_path=H5_FILE_PATH, 
    resampled_days_interval=7,
    )

In [ ]:
df_data_all.head(3)

In [ ]:
df_data_all.memory_usage(deep=True).sum()

#### Check out the data for one parcel

In [ ]:
# # Pick the first row
# example_row = df_data_all.iloc[0]
# parcel_ID = example_row.name

# # Get the corresponding label
# label_code = df_labels_all.loc[parcel_ID]['crpgrpc']
# label_name = df_labels_all.loc[parcel_ID]['crpgrpn']

# print('{} grows on parcel {}'.format(label_name, parcel_ID))

In [ ]:
# example_row_np = example_row.to_numpy()
# example_row_np = np.stack(example_row_np, axis=0)

# plt.rcParams['figure.figsize'] = [15, 8]
# plt.plot(common_days, example_row_np)
# # plt.legend(bands)
# plt.style.use('_classic_test_patch')
# plt.xlabel('day of year')
# plt.ylabel('channel value')
# plt.title(f'Data for parcel id {parcel_ID}')
# plt.grid()

#### Load geojson

In [ ]:
def load_geometry_dict_by_parcelid_all(region_names):
    geometry_dict_by_parcelid_all = {}
    for region_name in tqdm(region_names):
        region_name = region_name.strip('/')
        GEO_JSON_FILE_PATH = os.path.join(ROOT_DATA_PATH, f'GeoJSONs_regional_split/train/AT/demo_eurocrops_{region_name}.geojson')    

        with open(GEO_JSON_FILE_PATH, 'r') as file:
            geojson_data = json.load(file)

        geometry_dict_by_parcelid = {feature['properties']['recno']: feature['geometry'] 
                                     for feature in geojson_data['features']}
        geometry_dict_by_parcelid_all.update(geometry_dict_by_parcelid)
    return geometry_dict_by_parcelid_all


geometry_dict_by_parcelid_all = load_geometry_dict_by_parcelid_all(region_names=region_names)


#### Check crop types in the current dataset

In [ ]:
def get_crop_types_counts_and_ids(df_data_all, df_labels_all):
    crop_types_counts = {}
    crop_types_ids = {}

    regions_id_set = set(df_data_all.index)
    
    for i, region_id in enumerate(df_labels_all.index):
        if region_id not in regions_id_set:
            continue

        crop_name = df_labels_all.iloc[i]['crpgrpn']
        current_count = crop_types_counts.get(crop_name, 0) 
        crop_types_counts[crop_name] = current_count + 1

        if crop_name not in crop_types_ids:
            crop_types_ids[crop_name] = []
        crop_types_ids[crop_name].append(region_id)
    
    return crop_types_counts, crop_types_ids


        
crop_types_counts, crop_types_ids = get_crop_types_counts_and_ids(df_data_all=df_data_all, df_labels_all=df_labels_all)

print(f'Total crop fields: {sum(crop_types_counts.values())}')
crop_types_counts = {k: v for k, v in sorted(crop_types_counts.items(), key=lambda item: -item[1])}
crop_types_counts

In [ ]:
len(df_labels_all), len(df_data_all), len(geometry_dict_by_parcelid_all)

In [ ]:
def get_data_for_crop_type(crop_name):
    data = np.zeros(shape=(len(crop_types_ids[crop_name]), len(common_days), NUMBER_OF_CHANNELS), dtype=float)
    for i, region_id in enumerate(crop_types_ids[crop_name]):
        region_data = df_data_all.loc[region_id].to_numpy()
        data[i, ...] = np.stack(region_data)
    return data


In [ ]:
# @dataclass
# class CropNdviData:
#     mean: np.ndarray
#     std: np.ndarray

        
def calc_ndvi(B4, B8):
    return (B8 - B4) / (B8 + B4)
        
    
# def get_ndvi_data(data_crop) -> CropNdviData:
#     """
#     data_crop: [fields (for the crop), time (common_days), channels (bands B0-B12)]')
#     return: mean and std for ndvi "channel"
#     """
    
#     # B8-B4 / (B8+B4)   ( counting from B1 to B13)
#     B4 = data_crop[:, :, 4-1]
#     B8 = data_crop[:, :, 8-1]

#     data_crop_ndvi = calc_ndvi(B4, B8)
#     data_crop_mean_ndvi = np.mean(data_crop_ndvi, axis=0)
#     data_crop_std_ndvi = np.std(data_crop_ndvi, axis=0) 
    
#     return CropNdviData(mean=data_crop_mean_ndvi, std=data_crop_std_ndvi)
    

#### Flatten the data and labels

In [ ]:
data_flatten_list = []  
data_flatten_ndvi_list = []


for index, row in df_data_all.iterrows():
    region_data = row.to_numpy()
    region_data_stacked = np.stack(region_data)
    region_data_stacked_flat = region_data_stacked.flatten('F')  # so the first channel through time is continous
    data_flatten_list.append(region_data_stacked_flat)

    B4 = region_data_stacked[:, 4-1]
    B8 = region_data_stacked[:, 8-1]
    region_data_ndvi = calc_ndvi(B4, B8)
    data_flatten_ndvi_list.append(region_data_ndvi)
    
data_flatten = np.stack(data_flatten_list)
data_ndvi_flatten = np.stack(data_flatten_ndvi_list)

In [ ]:
data_flatten.shape, data_ndvi_flatten.shape

In [ ]:
df_labels_all

In [ ]:
all_crop_types = set(df_labels_all.iloc[:, 1].to_numpy())
data_flatten_labels = df_labels_all.iloc[:, 1].to_numpy()
print(set(data_flatten_labels))

### Select data for processing

#### choose crop types

In [ ]:

# crops_to_use__names = [
#     #'pasture_meadow',
#     'grain_maize',
#     #'winter_common_wheat_and_spelt',
#     #'other_plants_harvested_green',
#     'winter_barley',
#     'vineyards',
#     'soya',
#     'sugar_beet',
#     'winter_triticale',
#     'winter_rye',
#     'leguminous_plants',
#     'sunflower_and_yellow_bloomer',
#     #'other_cereals_for_the_production_of_grain',
#     'millet',
#     'winter_rape',
#     #'fresh_vegetables_melons_and_strawberries',
#     'summer_barley',
#     #'fruit_of_temperate_climate_zones',
#     'potatoes',
#     'summer_oats',
#     'cucurbits',
#     #'other_dry_pulses',
#     'summer_durum_wheat',
#     #'arable_land_seed_and_seedlings',
#     'winter_durum_wheat',
#     #'aromatic_plants_medicinal_and_culinary_plants',
#     #'energy_crops',
#     #'summer_common_wheat_and_spelt',
#     #'temporary_grass',
#     #'other_oil_seed_crops',
#     'hemp',
#     'nuts',
#     #'flowers_and_ornamental_plants',
#     #'berry_species',
#     #'nurseries',
# ]
crops_to_use__names = [ 
#     'sunflower_and_yellow_bloomer',
#     #'nuts',
#     'soya',
#     'vineyards',
#     'potatoes',
#     'summer_barley',
    
#     'cucurbits',
#     'sugar_beet',
#     'millet',
#     'grain_maize',
    
    'vineyards',
    'sunflower_and_yellow_bloomer',
    #'nuts',
    'sugar_beet',
    'nuts',
    'hemp',
    'summer_durum_wheat',
    'hops',
    
    
]

assert(len(crops_to_use__names) == len(set(crops_to_use__names)))

In [ ]:
indexes_to_use = []
for i in range(len(df_labels_all)):
    label = data_flatten_labels[i]
    if label not in crops_to_use__names:
        continue  # skip this data, not interested
    indexes_to_use.append(i)
# indexes_to_use = np.array(indexes_to_use)

selected_data_flatten_labels = np.take(data_flatten_labels, indexes_to_use)
selected_data_flatten = data_flatten[indexes_to_use, :]
selected_data_ndvi_flatten = data_ndvi_flatten[indexes_to_use, :]
len(set(selected_data_flatten_labels))

In [ ]:
selected_data_flatten_labels.shape, selected_data_flatten.shape, selected_data_ndvi_flatten.shape

In [ ]:
selected_data_flatten_labels

In [ ]:
data_ndvi_flatten[[1,2,3,], :].shape

#### Remap output classes to consecutive values


In [ ]:

selected_data_flatten_labels_mapping = {value: i for i, value in enumerate(crops_to_use__names)}

selected_data_flatten_labels_mapped = np.full(shape=selected_data_flatten_labels.shape, fill_value=1, dtype=int)
for i in range(len(selected_data_flatten_labels_mapped)):
    selected_data_flatten_labels_mapped[i] = selected_data_flatten_labels_mapping[selected_data_flatten_labels[i]]

In [ ]:
selected_data_flatten_labels_mapping

In [ ]:
len(set(selected_data_flatten_labels_mapped))

#### Determine the data type to use (for further processing)
And split dataset

In [ ]:
x = selected_data_ndvi_flatten

y = selected_data_flatten_labels_mapped
# print(len(x))
NUMBER_OF_CLASSES = len(set(y))


def unison_shuffled_copies(a, b):
    assert len(a) == len(b)
    p = np.random.permutation(len(a))
    return a[p], b[p]

n1, n2 = int(len(y) * 0.8), int(len(y) * 0.9)
x_shuffled, y_shuffled = unison_shuffled_copies(x, y)
x_train, x_valid, x_test = x_shuffled[:n1], x_shuffled[n1:n2], x_shuffled[n2:]
y_train, y_valid, y_test = y_shuffled[:n1], y_shuffled[n1:n2], y_shuffled[n2:]


len(x_train), len(x_valid), len(x_test)

#### Classes distribution and weights

In [ ]:
_, counts = np.unique(y_test, return_counts=True)
random_guess_accuraccy = max(counts) / sum(counts)
print(f'{random_guess_accuraccy = :.3f}')


counts_by_class = Counter(y_train)
# weights_for_classes = np.array([counts_by_class[i] / len(y_train) for i in range(NUMBER_OF_CLASSES)])  # the weights must sum up to 1?
weights_for_classes = np.array([1 / (counts_by_class[i]) for i in range(NUMBER_OF_CLASSES)])  # the weights must sum up to 1?
weights_for_classes = weights_for_classes / (weights_for_classes.sum())

weights_for_classes

## Data processing

In [ ]:
USE_CLASS_WEIGHTS = True

### t-SNE

In [ ]:
PLOT_TSNE_NDVI = False


if PLOT_TSNE_NDVI:
    import seaborn as sns
    from sklearn.manifold import TSNE

    tsne = TSNE(n_components=2, verbose=1, random_state=123)
    z = tsne.fit_transform(x)

    df = pd.DataFrame()
    df["y"] = selected_data_flatten_labels
    df["comp-1"] = z[:, 0]
    df["comp-2"] = z[:, 1]

    sns.scatterplot(x="comp-1", y="comp-2", hue=df.y.tolist(),
                    palette=sns.color_palette("hls", len(set(selected_data_flatten_labels))),
                    data=df).set(title="selected_data_flatten_labels crops")

In [ ]:
if PLOT_TSNE_NDVI:
    import seaborn as sns
    from sklearn.manifold import TSNE

    tsne = TSNE(n_components=3, verbose=1, random_state=123)
    z = tsne.fit_transform(x)


    fig = plt.figure(figsize=(3, 3))
    ax = fig.add_subplot(projection='3d')
    ax.scatter(z[:, 0], z[:, 1], z[:, 2])


In [ ]:
if PLOT_TSNE_NDVI:
    fig = px.scatter_3d(
        z, x=0, y=1, z=2,
        color=y, labels={'color': 'species'}
    )
    fig.update_traces(marker_size=8)
    fig.show()

### SVM

In [ ]:
from sklearn import svm

clf = svm.SVC(class_weight='balanced' if USE_CLASS_WEIGHTS else None)
clf.fit(x_train, y_train)

y_predicted = clf.predict(x_test)

predicted_correctly = np.count_nonzero(y_predicted == y_test)
predicted_correctly, len(y_predicted)

In [ ]:
def draw_prediction_matrix_for_test_data(y_predicted_labels, y_test_labels, normalize='true'):
    svm_ndvi_confusion_matrix = sklearn.metrics.confusion_matrix(y_true=y_test_labels, y_pred=y_predicted_labels, 
                                                                 normalize=normalize)
    disp = sklearn.metrics.ConfusionMatrixDisplay(svm_ndvi_confusion_matrix, display_labels=crops_to_use__names)
    disp.plot()
    
    
    predicted_correctly = np.count_nonzero(y_predicted_labels == y_test_labels)

    accuraccy = predicted_correctly / len(y_test_labels)

    # https://en.wikipedia.org/wiki/Cohen%27s_kappa
    kappa = 1 - (1 - accuraccy) / (1 - random_guess_accuraccy)

    print(f'svm_ndvi_kappa={kappa:.3f}, accuraccy={accuraccy:.3f}')
    

draw_prediction_matrix_for_test_data(y_predicted, y_test)
draw_prediction_matrix_for_test_data(y_predicted, y_test, normalize=None)

### NN linear

In [ ]:
NUMBER_OF_INPUTS = x[0].shape[0]
print(f'NUMBER_OF_INPUTS = {NUMBER_OF_INPUTS}, NUMBER_OF_CLASSES={NUMBER_OF_CLASSES}')


In [ ]:
model = nn.Sequential(
    nn.Linear(in_features=NUMBER_OF_INPUTS, out_features=200),
    nn.BatchNorm1d(num_features=200),  # batchnorm before activation
    nn.ReLU(),

    nn.Linear(in_features=200, out_features=100),
    nn.BatchNorm1d(num_features=100),
    nn.ReLU(),

    nn.Linear(in_features=100, out_features=50),
    nn.BatchNorm1d(num_features=50),
    nn.ReLU(),

    # nn.Dropout(0.05),
    nn.Linear(in_features=50, out_features=NUMBER_OF_CLASSES),
    nn.Softmax(dim=-1),  # not 0!
)

In [ ]:
# with torch.no_grad():
#     o = model(torch.Tensor(x_train[0:2]))
# o
# np.sum(o.numpy())

In [ ]:
class CropDataset(torch.utils.data.Dataset):
    def __init__(self, x, y):
        self.x = x
        self.y = y
        assert len(x) == len(y)
    
    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            raise Exception("Not supported")
        return self.x[idx].astype(np.float32), self.y[idx]


batch_size = 512  # 64
trainloader = torch.utils.data.DataLoader(CropDataset(x_train, y_train), batch_size=batch_size, shuffle=True)
validloader = torch.utils.data.DataLoader(CropDataset(x_valid, y_valid), batch_size=batch_size, shuffle=True)
testloader = torch.utils.data.DataLoader(CropDataset(x_test, y_test), batch_size=batch_size, shuffle=True)

next(iter(trainloader))[0].shape
trainloader.batch_size

In [ ]:
def get_predicted_and_true_labels(loader):
    y_predicted_labels = []
    y_test_labels = []

    with torch.no_grad():
        model.train(False)
        for data in loader:
            inputs, labels = data
            # calculate outputs by running images through the network
            outputs = model(inputs)
            label_predicted = np.argmax(outputs.numpy(), axis=1)
            y_predicted_labels.append(label_predicted)
            y_test_labels.append(labels)


    y_test_labels = np.concatenate(y_test_labels)
    y_predicted_labels = np.concatenate(y_predicted_labels)

    predicted_correctly = np.count_nonzero(y_predicted_labels == y_test_labels)
    accuraccy = predicted_correctly / len(y_test_labels)
    print(f'{accuraccy = :.3f}')
    return y_predicted_labels, y_test_labels, accuraccy


In [ ]:
for layer in model.children():
   if hasattr(layer, 'reset_parameters'):
       layer.reset_parameters()
       
train_loss_vec = []
valid_loss_vec = []
accuraccy_vec = []
lr_vec = []

In [ ]:
from torch import optim

if USE_CLASS_WEIGHTS:
    criterion = nn.CrossEntropyLoss(weight=torch.from_numpy(weights_for_classes.astype(np.float32)))
else:
    criterion = nn.CrossEntropyLoss()


# optimizer = optim.SGD(model.parameters(), lr=0.01)

# optimizer = optim.SGD(model.parameters(), lr=0.01)  # with batch_size 512

optimizer = optim.Adagrad(model.parameters(), lr=0.01)


scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min')

for epoch in range(60):  # 40
    for phase in ['train', 'valid']:
        running_loss = 0.0
        
        if phase == 'train':
            model.train(True)  # Set trainind mode = true
            dataloader = trainloader
        else:
            model.train(False)  # Set model to evaluate mode
            dataloader = validloader
        
        for i, data in enumerate(dataloader):
            inputs, labels = data

            if phase == 'train':
                optimizer.zero_grad()  # zero the parameter gradients
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
            else:
                with torch.no_grad():
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)
                
            running_loss += loss.item()
        
        if phase == 'train':
            train_loss = running_loss / len(dataloader.dataset)
            train_loss_vec.append(train_loss)
        else:
            valid_loss = running_loss / len(dataloader.dataset)
            valid_loss_vec.append(valid_loss)

   
    current_lr = optimizer.param_groups[0]['lr']
    scheduler.step(valid_loss)
    print(f'[{epoch + 1}]\t train loss: {train_loss:.6f}\t valid loss: {valid_loss:.6f}, {current_lr = }')
    _, _, accuraccy = get_predicted_and_true_labels(validloader)
    accuraccy_vec.append(accuraccy)
    lr_vec.append(current_lr)
    running_loss = 0.0

print('Finished Training')

In [ ]:
plt.plot(train_loss_vec)
plt.plot(valid_loss_vec)
plt.show()

In [ ]:
plt.plot(accuraccy_vec)

In [ ]:
plt.plot(lr_vec)

In [ ]:
y_predicted_labels_test, y_test_labels, _ = get_predicted_and_true_labels(testloader)
draw_prediction_matrix_for_test_data(y_predicted_labels_test, y_test_labels)
draw_prediction_matrix_for_test_data(y_predicted_labels_test, y_test_labels, normalize=None)